In [22]:
import os
from ultralytics import YOLO
import cv2

In [23]:
model = YOLO(r'C:\Users\BM MONEY\runs\detect\yolov11_finetune\weights\best.pt')

In [24]:
input_path = 'videoplayback.mp4'

In [25]:
cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise FileNotFoundError(f"Could not open video: {input_path}")

In [26]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [27]:
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError(f"Could not create output video: {output_path}")

In [28]:
# Track objects with IDs
no_helmet_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run inference with tracking
    results = model.track(frame, conf=0.5, tracker="botsort.yaml", persist=True)
    
    # Create a copy of the frame for custom annotations
    annotated_frame = frame.copy()
    
    # Update no-helmet IDs
    current_no_helmet_ids = set()
    for box in results[0].boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        track_id = int(box.id) if box.id is not None else None
        if track_id is None:
            continue
        
        # Get original head bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        # Expand bounding box to cover body (for riders with/without helmets)
        head_width = x2 - x1
        head_height = y2 - y1
        body_height = int(head_height * 4)  # Assume body is ~4x head height
        body_width = int(head_width * 2)    # Assume body is ~2x head width
        
        # New coordinates
        new_x1 = max(0, x1 - head_width // 2)  # Extend left
        new_x2 = min(width, x2 + head_width // 2)  # Extend right
        new_y1 = max(0, y1)  # Keep top at head
        new_y2 = min(height, y1 + body_height)  # Extend downward
        
        # Draw bounding box
        color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Green for With Helmet, Red for Without Helmet
        cv2.rectangle(annotated_frame, (new_x1, new_y1), (new_x2, new_y2), color, 2)
        label_text = f"ID: {track_id} {'No Helmet' if cls == 1 else 'Helmet'}"
        cv2.putText(annotated_frame, label_text, (new_x1, new_y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Handle non-helmeted riders (class ID 1)
        if cls == 1 and conf > 0.5:
            print(f"Person without helmet detected! Track ID: {track_id}")
            current_no_helmet_ids.add(track_id)
            
            # Draw transparent red block over the body for non-helmeted riders
            red_block_alpha = 0.4
            overlay = annotated_frame.copy()
            cv2.rectangle(overlay, (new_x1, new_y1), (new_x2, new_y2), (0, 0, 255), -1)  # Filled red
            cv2.addWeighted(overlay, red_block_alpha, annotated_frame, 1 - red_block_alpha, 0, annotated_frame)
    
    # Global warning for non-helmeted riders
    if current_no_helmet_ids:
        cv2.putText(annotated_frame, "No Helmet Detected!", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Write frame to output video
    out.write(annotated_frame)
    
    # Display frame
    cv2.imshow("Helmet Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Output video saved at: {output_path}")


0: 384x640 (no detections), 63.3ms
Speed: 4.6ms preprocess, 63.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.3ms
Speed: 4.2ms preprocess, 59.3ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.6ms
Speed: 5.4ms preprocess, 59.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.1ms
Speed: 4.3ms preprocess, 59.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.2ms
Speed: 2.1ms preprocess, 43.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.6ms
Speed: 1.6ms preprocess, 30.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.8ms
Speed: 1.7ms preprocess, 25.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.9ms
Speed: 2.0ms preprocess, 30.9ms i